In [1]:
import cv2
import numpy as np
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import pandas as pd

def cloud_cover(filename, start, end):
    ###
    # INPUTS: 
    # * filename (string): the name of the image containing the cloud cover data from meteoblue.com
    # * start (datetime object): the start datetime of the data in the image
    # * end (datetime object): the end datetime of the data in the image
    # 
    # OUTPUTS:
    # * dataframe containing cloud cover data (as a percentage)
    ###

    # Load the image
    image_path = filename
    image = cv2.imread(image_path)

    # Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the yellow color in HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Create a mask to isolate the yellow region
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Bitwise AND the original image with the yellow mask to get the yellow region
    yellow_region = cv2.bitwise_and(image, image, mask=yellow_mask)

    # Convert the yellow region to grayscale
    gray_yellow_region = cv2.cvtColor(yellow_region, cv2.COLOR_BGR2GRAY)

    # Threshold the grayscale yellow region to create a binary mask
    _, yellow_mask_binary = cv2.threshold(gray_yellow_region, 1, 255, cv2.THRESH_BINARY)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(yellow_mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Identify the contour of the relevant plot (assumed to be the largest contour)
    relevant_contour = max(contours, key=cv2.contourArea)

    # Get the bounding box of the relevant plot
    x, y, w, h = cv2.boundingRect(relevant_contour)

    # Crop the image to focus on the relevant plot
    relevant_plot = image[y:y+h, 157:2016]

    # Calculate the time interval between pixels
    time_interval = (end_time - start_time) / relevant_plot.shape[1]

    # Define a function to extract the cloud cover from a pixel color
    def get_cloud_cover(pixel):
        # Check if the pixel is yellow (sunshine)
        if pixel[0]==80 and pixel[1]==224 and pixel[2]==248:
            return 0
        # If not yellow, assume it's gray (clouds) and calculate cloud cover based on the intensity
        gray_intensity = pixel[0]
        cloud_cover = (255 - gray_intensity) / 255 * 100
        return cloud_cover

    cloud_cover_data = []

    # Interpolate cloud cover for each hour
    for hour in range(24*(end_time-start_time).days):  # Assuming 24 hours in a day
        # Calculate the target time for the current hour
        target_time = start_time + timedelta(hours=hour)

        # Find the closest pixel in time
        col = int((target_time - start_time) / time_interval)

        # Get the BGR values of the pixel
        pixel = relevant_plot[0, col]

        # Calculate the cloud cover for the pixel
        cloud_cover = get_cloud_cover(pixel)

        # Append the time and cloud cover to the data
        cloud_cover_data.append((target_time, cloud_cover))
    
    pd.set_option('display.max_rows', len(cloud_cover_data))
    return pd.DataFrame(cloud_cover_data,columns=['Datetime','Cloud cover (%)']).set_index('Datetime')

In [2]:
# The start and end datetime in the plot
start_time = datetime(2022, 7, 16, 0, 0, 0, tzinfo=timezone(timedelta(hours=3)))  # Adjusted for UTC+03:00
end_time = datetime(2022, 8, 1, 0, 0, 0, tzinfo=timezone(timedelta(hours=3)))  # Adjusted for UTC+03:00
cloud_cover(filename='koropi_jul_16-31_22_weather_history_simple_hd.png', start=start_time, end=end_time)

,Cloud cover (%)
Datetime,
2022-07-16 00:00:00+03:00,0
2022-07-16 01:00:00+03:00,0
2022-07-16 02:00:00+03:00,0
2022-07-16 03:00:00+03:00,0
2022-07-16 04:00:00+03:00,0
2022-07-16 05:00:00+03:00,0
2022-07-16 06:00:00+03:00,0
2022-07-16 07:00:00+03:00,0
2022-07-16 08:00:00+03:00,0
